In [ ]:
import wandb
import pandas as pd
import json

In [ ]:
# Initialize a W&B API object
api = wandb.Api()

# Replace with your project and entity name
entity = "shunyaist"
project = "GQCO"

# Fetch the runs from the project
runs = api.runs(f"{entity}/{project}")

In [121]:
csv_files = []

for run in runs:
    if 'main' in run.tags or 'tune' in run.tags:
        metrics = run.history(samples=1000000)
        metrics = metrics[['current_size', 'trainer/global_step', '_runtime']]
        metrics.rename(columns={'current_size': 'Size'}, inplace=True)
        metrics.rename(columns={'trainer/global_step': 'EpochSum'}, inplace=True)
        metrics.rename(columns={'_runtime': 'TimeSum'}, inplace=True)
        metrics['WandB-ID'] = run.id
        metrics['Size'] = metrics['Size'].apply(lambda x: round(x) if pd.notnull(x) else x).astype('Int64', errors='ignore')
        metrics['ABCI-ID'] = json.loads(run.json_config)['job_id']['value']
        metrics['#GPU'] = json.loads(run.json_config)['world_size']['value']
        metrics['LogFreqAcc'] = json.loads(run.json_config)['log_freq_acc']['value']
        metrics['InitCheckpoint'] = json.loads(run.json_config)['init_checkpoint']['value']
        metrics['TuneSize'] = json.loads(run.json_config)['tune_size']['value']

        save_name = f"output/{run.id}_metrics.csv"
        metrics.to_csv(save_name, index=False)
        csv_files.append(save_name)

df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)


In [122]:
# Remove rows where current_size is None
df_cleaned = df.dropna(subset=['Size'])

# Retain rows where (trainer/global_step + 1) % LogFreqAcc == 0
df_cleaned.loc[:, 'EpochSum'] = df_cleaned['EpochSum'] + 1
df_filtered = df_cleaned[(df_cleaned['EpochSum']) % df_cleaned['LogFreqAcc'] == 0].copy()

df_filtered.loc[:, '#Node'] = df_filtered['#GPU'] / 4

df_filtered = df_filtered[['ABCI-ID', 'WandB-ID', '#Node', '#GPU', 'Size', 'TuneSize', 'EpochSum', 'TimeSum', 'InitCheckpoint']]

df_filtered.to_csv('output/wandb_result.csv')